<a href="https://colab.research.google.com/github/alfcastillo90/Algorithms/blob/master/pyspark_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
## descargamos e instalamos jdk y apache spark
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 
import os 
import findspark 
## creamos las variables de entorno JAVA_HOME y SPARK_HOME
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'

In [ ]:
## inicializamos
findspark.init()

In [76]:
## nos integramos con google drive
from google.colab import drive 
drive.mount("/content/gdrive")
gdrive='content/gdrive/MyDrive/Estudios/UBO/BIGDATA'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
## creamos la carpeta streaming, primero validamos si existe o no
!test -d /content/streaming && echo 'la carpeta existe' || mkdir streaming

la carpeta existe


In [ ]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.master('local').appName('PySparkStreaming').config('spark.ui.port',4050).getOrCreate()

In [77]:
## Definimos el esquema

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

structureSchema = StructType([ \
                     StructField('index', IntegerType(), True), \
                     StructField('Order ID', StringType(), True), \
                     StructField('Date', StringType(), True), \
                     StructField('Status', StringType(), True), \
                     ])


In [78]:
## Leemos el archivo csv y le asignamos el esquema que fue definido previamente
dataFrame = spark.read.csv('/'+gdrive+'/amazon-orders-status.csv', sep=',', header=True, schema=structureSchema)


In [79]:
dataFrame.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)



In [80]:
streamingDataFrame = spark.readStream.format('csv').schema(structureSchema).option('header', True).load('streaming')

In [81]:
streamingDataFrame.isStreaming

True

In [85]:
## dataFrame.createOrReplaceTempView('data');
dataFrame2 = dataFrame.filter((dataFrame['index'] > 100) & (dataFrame['index'] < 105))
##df2 =  dataFrame.select('SELECT * FROM data where index >= 100 and index <=105').show() ##spark.sql('SELECT * FROM data where index >= 100 and index <=105').show()



+-----+-------------------+--------+--------------------+
|index|           Order ID|    Date|              Status|
+-----+-------------------+--------+--------------------+
|    0|405-8078784-5731545|04-30-22|           Cancelled|
|    1|171-9198151-1101146|04-30-22|Shipped - Deliver...|
|    2|404-0687676-7273146|04-30-22|             Shipped|
|    3|403-9615377-8133951|04-30-22|           Cancelled|
|    4|407-1069790-7240320|04-30-22|             Shipped|
|    5|404-1490984-4578765|04-30-22|             Shipped|
|    6|408-5748499-6859555|04-30-22|             Shipped|
|    7|406-7807733-3785945|04-30-22|Shipped - Deliver...|
|    8|407-5443024-5233168|04-30-22|           Cancelled|
|    9|402-4393761-0311520|04-30-22|             Shipped|
|   10|407-5633625-6970741|04-30-22|             Shipped|
|   11|171-4638481-6326716|04-30-22|             Shipped|
|   12|405-5513694-8146768|04-30-22|Shipped - Deliver...|
|   13|408-7955685-3083534|04-30-22|             Shipped|
|   14|408-129

In [86]:
import time
import pandas as pd 
import numpy as np

for n in range(0, dataFrame2.count()):
  df = pd.DataFrame(dataFrame.filter((dataFrame['index'] >= n) & (dataFrame['index'] <= n+10)).toPandas())
  display(df)


##pd.DataFrame(dataFrame2.toPandas())


,index,Order ID,Date,Status
0,0,405-8078784-5731545,04-30-22,Cancelled
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer
2,2,404-0687676-7273146,04-30-22,Shipped
3,3,403-9615377-8133951,04-30-22,Cancelled
4,4,407-1069790-7240320,04-30-22,Shipped
5,5,404-1490984-4578765,04-30-22,Shipped
6,6,408-5748499-6859555,04-30-22,Shipped
7,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer
8,8,407-5443024-5233168,04-30-22,Cancelled
9,9,402-4393761-0311520,04-30-22,Shipped


,index,Order ID,Date,Status
0,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer
1,2,404-0687676-7273146,04-30-22,Shipped
2,3,403-9615377-8133951,04-30-22,Cancelled
3,4,407-1069790-7240320,04-30-22,Shipped
4,5,404-1490984-4578765,04-30-22,Shipped
5,6,408-5748499-6859555,04-30-22,Shipped
6,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer
7,8,407-5443024-5233168,04-30-22,Cancelled
8,9,402-4393761-0311520,04-30-22,Shipped
9,10,407-5633625-6970741,04-30-22,Shipped


,index,Order ID,Date,Status
0,2,404-0687676-7273146,04-30-22,Shipped
1,3,403-9615377-8133951,04-30-22,Cancelled
2,4,407-1069790-7240320,04-30-22,Shipped
3,5,404-1490984-4578765,04-30-22,Shipped
4,6,408-5748499-6859555,04-30-22,Shipped
5,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer
6,8,407-5443024-5233168,04-30-22,Cancelled
7,9,402-4393761-0311520,04-30-22,Shipped
8,10,407-5633625-6970741,04-30-22,Shipped
9,11,171-4638481-6326716,04-30-22,Shipped


,index,Order ID,Date,Status
0,3,403-9615377-8133951,04-30-22,Cancelled
1,4,407-1069790-7240320,04-30-22,Shipped
2,5,404-1490984-4578765,04-30-22,Shipped
3,6,408-5748499-6859555,04-30-22,Shipped
4,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer
5,8,407-5443024-5233168,04-30-22,Cancelled
6,9,402-4393761-0311520,04-30-22,Shipped
7,10,407-5633625-6970741,04-30-22,Shipped
8,11,171-4638481-6326716,04-30-22,Shipped
9,12,405-5513694-8146768,04-30-22,Shipped - Delivered to Buyer


,index,Order ID,Date,Status
0,4,407-1069790-7240320,04-30-22,Shipped
1,5,404-1490984-4578765,04-30-22,Shipped
2,6,408-5748499-6859555,04-30-22,Shipped
3,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer
4,8,407-5443024-5233168,04-30-22,Cancelled
5,9,402-4393761-0311520,04-30-22,Shipped
6,10,407-5633625-6970741,04-30-22,Shipped
7,11,171-4638481-6326716,04-30-22,Shipped
8,12,405-5513694-8146768,04-30-22,Shipped - Delivered to Buyer
9,13,408-7955685-3083534,04-30-22,Shipped


...

,index,Order ID,Date,Status
0,14,408-1298370-1920302,04-30-22,Shipped - Delivered to Buyer
1,15,403-4965581-9520319,04-30-22,Shipped - Delivered to Buyer
2,16,406-9379318-6555504,04-30-22,Shipped
3,17,405-9013803-8009918,04-30-22,Shipped
4,18,402-4030358-5835511,04-30-22,Shipped - Delivered to Buyer
5,19,405-5957858-1051546,04-30-22,Shipped
6,20,405-0607769-0716360,04-30-22,Shipped
7,21,404-8494550-5860325,04-30-22,Shipped
8,22,171-1305077-2813934,04-30-22,Shipped
9,23,404-6019946-2909948,04-30-22,Cancelled


,index,Order ID,Date,Status
0,15,403-4965581-9520319,04-30-22,Shipped - Delivered to Buyer
1,16,406-9379318-6555504,04-30-22,Shipped
2,17,405-9013803-8009918,04-30-22,Shipped
3,18,402-4030358-5835511,04-30-22,Shipped - Delivered to Buyer
4,19,405-5957858-1051546,04-30-22,Shipped
5,20,405-0607769-0716360,04-30-22,Shipped
6,21,404-8494550-5860325,04-30-22,Shipped
7,22,171-1305077-2813934,04-30-22,Shipped
8,23,404-6019946-2909948,04-30-22,Cancelled
9,24,402-3384087-4005164,04-30-22,Shipped


,index,Order ID,Date,Status
0,16,406-9379318-6555504,04-30-22,Shipped
1,17,405-9013803-8009918,04-30-22,Shipped
2,18,402-4030358-5835511,04-30-22,Shipped - Delivered to Buyer
3,19,405-5957858-1051546,04-30-22,Shipped
4,20,405-0607769-0716360,04-30-22,Shipped
5,21,404-8494550-5860325,04-30-22,Shipped
6,22,171-1305077-2813934,04-30-22,Shipped
7,23,404-6019946-2909948,04-30-22,Cancelled
8,24,402-3384087-4005164,04-30-22,Shipped
9,25,405-8191138-5176316,04-30-22,Shipped - Delivered to Buyer


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored